# Responsible AI GUI

## Analysis of Synthetic Data

This demo illustrates a hypothetical scenario of how likely a programmer should be given access to a GPT2 model for inferencing, based on information such as their favorite programming language, preference for tabs vs spaces, OS, location and so forth. Each programmer will be given a score between [0,10] where a score between [7,10] indicates access given to the programmer and [0,7) indicates access denied. The data were synthetically generated via the [PyPI package, Fibber.io](https://pypi.org/project/fibber/).

First, we need to specify the version of the RAI components which are available in the workspace. This was specified when the components were uploaded, and will have defaulted to '1':

>[NOTE] Must use Python 3.10 SDK V2 for this demo.


In [ ]:
version_string = "1"

We also need to give the name of the compute cluster we want to use in AzureML. Later in this notebook, we will create it if it does not already exist:

In [ ]:
compute_name = "cpu-cluster"

Finally, we need to specify a version for the data and components we will create while running this notebook. This should be unique for the workspace, but the specific value doesn't matter:

In [ ]:
rai_programmer_example_version_string = "1"

## Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the MLClient from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace.

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
# ml_client = MLClient(
#     credential=credential,
#     subscription_id=subscription_id,
#     resource_group_name=resource_group,
#     workspace_name=workspace,
# )

ml_client = MLClient.from_config(
    credential=credential,
)
print(ml_client)

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)
print(ml_client_registry)

## Accessing the Data

We supply the synthetic data as a pair of parquet files and accompanying `MLTable` file. We can read them in and take a brief look:

In [ ]:
import os
import pandas as pd

Now define the paths to the data:

In [ ]:
train_data_path = "data/programmer-rai-data/train/"

In [ ]:
test_data_path = "data/programmer-rai-data/test/"

Load some data for a quick view:

In [ ]:
import mltable

tbl = mltable.load(train_data_path)
train_df: pd.DataFrame = tbl.to_pandas_dataframe()

# test dataset should have less than 5000 rows
test_df = mltable.load(test_data_path).to_pandas_dataframe()
assert len(test_df.index) <= 5000

display(train_df)

The (synthetic) data are about a collection of programmers, with a 'score' column which we wish to predict:

In [ ]:
target_column_name = "score"

First, we need to upload the datasets to our workspace.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_train_data = "Programmers_Train_MLTable"
input_test_data = "Programmers_Test_MLTable"

try:
    # Try getting data already registered in workspace
    train_data = ml_client.data.get(
        name=input_train_data, version=rai_programmer_example_version_string
    )
    test_data = ml_client.data.get(
        name=input_test_data, version=rai_programmer_example_version_string
    )
except Exception as e:
    # If no data of specified version exist, create new one
    train_data = Data(
        path=train_data_path,
        type=AssetTypes.MLTABLE,
        description="RAI programmers training data",
        name=input_train_data,
        version=rai_programmer_example_version_string,
    )
    ml_client.data.create_or_update(train_data)

    test_data = Data(
        path=test_data_path,
        type=AssetTypes.MLTABLE,
        description="RAI programmers test data",
        name=input_test_data,
        version=rai_programmer_example_version_string,
    )
    ml_client.data.create_or_update(test_data)

# Creating the Model

To simplify the model creation process, we're going to use a pipeline.

We create a directory for the training script:

In [ ]:
import os

os.makedirs("register_model_src", exist_ok=True)
os.makedirs("programmer_component_src", exist_ok=True)

Next, we write out our training script:

In [ ]:
%%writefile programmer_component_src/training_script_reg.py

import argparse
import os
import shutil
import tempfile


from azureml.core import Run

import mlflow
import mlflow.sklearn

import mltable

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", type=str, help="Path to training data")
    parser.add_argument("--target_column_name", type=str, help="Name of target column")
    parser.add_argument("--model_output", type=str, help="Path of output model")

    # parse args
    args = parser.parse_args()

    # return args
    return args

def create_regression_pipeline(X, y):
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])

    # Append classifier to preprocessing pipeline.
    # Now we have a full prediction pipeline.
    pipeline = Pipeline(steps=[('preprocessor', feat_pipe),
                               ('model', LinearRegression())])
    return pipeline.fit(X, y)

def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)
    
    # Read in data
    print("Reading data")
    tbl = mltable.load(args.training_data)
    all_data = tbl.to_pandas_dataframe()

    print("Extracting X_train, y_train")
    print("all_data cols: {0}".format(all_data.columns))
    y_train = all_data[args.target_column_name]
    X_train = all_data.drop(labels=args.target_column_name, axis="columns")
    print("X_train cols: {0}".format(X_train.columns))

    print("Training model")
    # The estimator can be changed to suit
    model = create_regression_pipeline(X_train, y_train)

    # Saving model with mlflow - leave this section unchanged
    with tempfile.TemporaryDirectory() as td:
        print("Saving model with MLFlow to temporary directory")
        tmp_output_dir = os.path.join(td, "my_model_dir")
        mlflow.sklearn.save_model(sk_model=model, path=tmp_output_dir)

        print("Copying MLFlow model to output path")
        for file_name in os.listdir(tmp_output_dir):
            print("  Copying: ", file_name)
            # As of Python 3.8, copytree will acquire dirs_exist_ok as
            # an option, removing the need for listdir
            shutil.copy2(src=os.path.join(tmp_output_dir, file_name), dst=os.path.join(args.model_output, file_name))


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

In [ ]:
%%writefile register_model_src/register.py

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------

import argparse
import json
import os
import time


from azureml.core import Run

import mlflow
import mlflow.sklearn

# Based on example:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-cli
# which references
# https://github.com/Azure/azureml-examples/tree/main/cli/jobs/train/lightgbm/iris


def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--model_input_path", type=str, help="Path to input model")
    parser.add_argument(
        "--model_info_output_path", type=str, help="Path to write model info JSON"
    )
    parser.add_argument(
        "--model_base_name", type=str, help="Name of the registered model"
    )
    parser.add_argument(
        "--model_name_suffix", type=int, help="Set negative to use epoch_secs"
    )

    # parse args
    args = parser.parse_args()

    # return args
    return args


def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)

    print("Loading model")
    mlflow_model = mlflow.sklearn.load_model(args.model_input_path)

    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix
    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    print(f"Registering model as {registered_name}")

    print("Registering via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=mlflow_model,
        registered_model_name=registered_name,
        artifact_path=registered_name,
    )

    print("Writing JSON")
    dict = {"id": "{0}:1".format(registered_name)}
    output_path = os.path.join(args.model_info_output_path, "model_info.json")
    with open(output_path, "w") as of:
        json.dump(dict, fp=of)


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Now, we can build this into an AzureML component:

In [ ]:
from azure.ai.ml import load_component

yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_programmers_training_component
display_name: Programmers training component for RAI example
version: {rai_programmer_example_version_string}
type: command
inputs:
  training_data:
    type: path
  target_column_name:
    type: string
outputs:
  model_output:
    type: path
code: ./programmer_component_src/
environment: azureml://registries/azureml/environments/AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu/versions/4
command: >-
  python training_script_reg.py
  --training_data ${{{{inputs.training_data}}}}
  --target_column_name ${{{{inputs.target_column_name}}}}
  --model_output ${{{{outputs.model_output}}}}
"""

yaml_filename = "ProgrammersRegTrainingComp.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents)

train_model_component = load_component(source=yaml_filename)

In [ ]:
yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: register_model
display_name: Register Model
version: {rai_programmer_example_version_string}
type: command
is_deterministic: False
inputs:
  model_input_path:
    type: path
  model_base_name:
    type: string
  model_name_suffix: # Set negative to use epoch_secs
    type: integer
    default: -1
outputs:
  model_info_output_path:
    type: path
code: ./register_model_src/
environment: azureml://registries/azureml/environments/AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu/versions/4
command: >-
  python register.py
  --model_input_path ${{{{inputs.model_input_path}}}}
  --model_base_name ${{{{inputs.model_base_name}}}}
  --model_name_suffix ${{{{inputs.model_name_suffix}}}}
  --model_info_output_path ${{{{outputs.model_info_output_path}}}}

"""

yaml_filename = "register.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents)

register_component = load_component(source=yaml_filename)

We need a compute target on which to run our jobs. The following checks whether the compute specified above is present; if not, then the compute target is created.

In [ ]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="Standard_D2_v2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600,
    )
    ml_client.compute.begin_create_or_update(my_compute)
    print("Initiated compute creation")

## Running a training pipeline

Now that we have our training component, we can run it. We begin by generating a unique name for the mode;

In [ ]:
import time

model_name_suffix = int(time.time())
model_name = "rai_programmer_example_reg"

Next, we define our training pipeline. This has two components. The first is the training component which we defined above. The second is a component to register the model in AzureML:

In [ ]:
from azure.ai.ml import dsl, Input

programmers_train_mltable = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{rai_programmer_example_version_string}",
    mode="download",
)
programmers_test_mltable = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{rai_programmer_example_version_string}",
    mode="download",
)


@dsl.pipeline(
    compute=compute_name,
    description="Register Model for RAI Programmers example",
    experiment_name=f"RAI_Programmers_Example_Model_Training_{model_name_suffix}",
)
def my_training_pipeline(target_column_name, training_data):
    trained_model = train_model_component(
        target_column_name=target_column_name, training_data=training_data
    )
    trained_model.set_limits(timeout=120)

    _ = register_component(
        model_input_path=trained_model.outputs.model_output,
        model_base_name=model_name,
        model_name_suffix=model_name_suffix,
    )

    return {}


model_registration_pipeline_job = my_training_pipeline(
    target_column_name, programmers_train_mltable
)

With the training pipeline defined, we can submit it for execution in AzureML. We define a helper function to wait for the job to complete:

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display


def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

## Create Responsible AI Insight Dashboard

Now, we will create a job within the Azure ML Studio with the no-code wizard experience. Since we already registered our model, we can now generate Responsible AI Insights wih the "**Responsible AI > Create dashboard**" button.

![Alt text](Media/create-rai-dashboard.png)

First, we need to pick a train and test dataset that we used to train and test your model.

![Alt text](Media/create-rai-dashboard-train-dataset.png)

![Alt text](Media/create-rai-dashboard-test-dataset.png)

Next, we will be chossing regression to match the model.

![Alt text](Media/create-rai-dashboard-regression-task.png)

For this scenario, we have a choice of chossing either the debugging profile or real-life intervention profile. For now, let's choose the **Model Debugging** profile.

![Alt text](Media/create-rai-dashboard-regression-debug.png)

Let's move forward with model debugging and customize the dashboard to include error analysis, counterfactual analysis, and model explanation. 

For the error analysis, we can select up to two features to pre-generate an error heat map for.

For the counterfactual analysis, let's investigate some examples (let's go with 10example per datapoint) where we automatically exercise features just enough, so we can get a score between 7 and 10. We can see which features are perturb if we don't want certain features to be changed.

![Alt text](Media/create-rai-dashboard-regression-parameters-debugging.png)

Once satisfied, we can now move on to the final step to configure our experiment. From here, we can provide a job name for our Responsible AI dashboard.

![Alt text](Media/create-rai-dashboard-regression-exp.png)
Now, we wait for the job to be completed.

![Alt text](Media/create-rai-dashboard-regression-exp-pending.png)

We can also check on the job's progress by clicking the "**View Job**" button.

![Alt text](Media/create-rai-dashboard-regression-exp-progress.png)

## Viewing the RAI Dashboard

We can now view the dashboard(s) by navigating to the registered model we've just created. This can be accomplished by clicking on the **Responsible AI** tab at the model's detail page. 

An alternative is to click on the **View** button after the job is completed. 

![Alt text](Media/create-rai-dashboard-regression-exp-completed.png)

We have enabled an intergration of our workspace compute resources to access all the features such as retraining error trees, recalculating probabilities and generating insights in real time. 

![Alt text](Media/rai-dashboard-regression-insights.png)

The different components of the Responsible AI dashboard are designed such that they can easily communicate with each other. You can create cohorts of your data to slice and dice your analysis and interactively pass cohorts and insights from one component to another for deep-dive investigations. You can hide the different components you’ve generated for the dashboard in the “dashboard configuration” or add them back by clicking the blue “plus” icon.

We first look at our error tree, which tells us where the distribution of most of our errors lie. It seems that our models made the greatest number of errors for programmers living in Antarctica who don’t program in C, PHP, or Swift and don’t contribute that often to GitHub repos. We can easily save this as a new cohort to investigate later, but in the meanwhile it will show up as a “Temporary cohort” in the subsequent components.

![Alt text](Media/rai-dashboard-regression-insights-generate2.png)

We can use the **data analysis** to see if feature distribution in our dataset is skewed. This can cause a model to incorrectly predict datapoints belonging to an underrepresented group or to be optimized along an inappropriate metric. If we bin our x-axis to be the ground truth of different scores a programmer can get (where 7-10 is the accepted range) and look at the style, we see that there is a highly skewed distribution of programmers who use tabs being scored lower and programmers who use spaces being scored higher.

![Alt text](Media/rai-dashboard-regression-insights-data-analysis-2.png)

Since we know our model made the most amount of error for those living in Antarctica, when we investigate location, we see a highly skewed distribution of programmers living in Antarctica who were scored lower. What this means is that our model will unfairly favor those who are using spaces, and not living in Antarctica when providing access to the application we built.

![Alt text](Media/rai-dashboard-regression-insights-data-analysis-location.png)

 For **feature importance**, we can see for our overall model, which features were the most important to the model’s predictions; and we can see that style (tabs or spaces) is by far the most considered, then operating system then programming language. If we click into style, we can see that using ‘spaces’ has a positive feature importance and ‘tabs’ has a negative feature importance showing us that ‘spaces’ is what contributes to a higher score.

 ![Alt text](Media/rai-dashboard-regression-insights-feature.png)

We can also look at two specific programmers who got a low and high score. **Row 35** has a high score and uses spaces and **row 2** has a low score and uses tabs. When we look at the individual feature importance of each programmers’ features, we can see that the ‘spaces’ positively contributed to **Row 35**’s high score, while ‘tabs’ contributed negatively towards a lower score for **Row 2**. 

![Alt text](Media/rai-dashboard-regression-insights-individual-feature.png)

We can take a deeper look with counterfactual what-if examples. When selecting someone below the 7 to 10 range prediction, we can see what bare minimum changes could happen to their features to lead to much higher predictions. In this programmer’s case, some recommended changes would be switching their style to spaces.

Let's select **Index 440** and create a what-if counterfactual.

![Alt text](Media/rai-dashboard-regression-insights-counterfactuals.png)

![Alt text](Media/rai-dashboard-regression-insights-counterfactuals-2.png)

Finally, if we wanted to purely use historic data to identify the features that have the most direct effect on our outcome of interest, in this case the score, we can use causal analysis.  

For that, we create a new Responsible Dashboard with **Real-Life Interventions** components.
To create a new dashboard, go to the same model and under Responsible AI, click on Responsible AI insights, click on Create Dashboard 

![Alt text](Media/responsible-ai-insights.png)

Follow the same steps followed before until where you can choose Real Life Interventions

![Alt text](Media/rai-dashboard-regression-real-life-interventions.png)

In our case, we want to understand the causal effect of years of experience and number of GitHub repos a programmer has contributed to on the score. 

![Alt text](Media/rai-dashboard-regression-real-life-interventions-2.png)

Be sure to use the same experiment for tracking purposes.

![Alt text](Media/rai-dashboard-regression-real-life-interventions-exp.png)

The aggregate causal effects show you overall for your whole dataset, on average, increasing the number of GitHub repos by 1 increases the score by 0.096 whereas increasing the number of years of experience by 1 doesn’t increase the score by much at all.

![Alt text](Media/rai-dashboard-regression-real-life-interventions-causal-analysis.png)

However, if we want to look at individual programmers and perturb those values and see the outcome of specific treatments to years of experience, we can see that for some programmers, increasing the years of experience does cause the score to increase by a bit.

![Alt text](Media/rai-dashboard-regression-real-life-interventions-causal-analysis-2.png)

Additionally, the treatment policy tab can help us decide what overall treatment policy to take to maximize real-world impact on our score.  We can see the best future interventions to apply to certain segmentations of our programmer population to see the biggest boost in the scores overall.

![Alt text](Media/rai-dashboard-regression-real-life-interventions-causal-analysis-3.png)

And if you can only focus on 10 programmers to reach out to, you can see a ranked list of top k programmers who would gain the most from either increasing or decreasing the number of GitHub repos.

![Alt text](Media/rai-dashboard-regression-real-life-interventions-causal-analysis-4.png)

## Create a RAI Scorecard with UI

You can create a scorecard for any Responsible AI dashboard you generate by clicking on **Create Responsible AI insights > Generate new PDF scorecard** which will open up a panel for you to walk through the same steps via a UI wizard but without any code.

![Alt text](Media/rai-dashboard-regression-PDF-scorecard.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-2.1.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-2.1.1.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-3.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-4.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-4.1.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-5.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-6.png)

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-7.1.png)

## Download the Responsible AI scorecard

To view your Responsible AI scorecard, go into your model registry and select the registered model you have generated a Responsible AI dashboard for. Once you click on your model, click on the **Responsible AI** tab to view a list of generated dashboards. Select which dashboard you’d like to export a Responsible AI scorecard PDF for by clicking **Create Responsible AI insights > View all PDF scorecards**.

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-pre-final.png)

Select which scorecard you’d like to download from the list and click Download to download the PDF to your machine.

![Alt text](Media/rai-dashboard-regression-PDF-scorecard-generate-final.png)

## Learn how to read the Responsible AI scorecard

The Responsible AI scorecard is a PDF summary of your key insights from the Responsible AI dashboard. The first summary segment of the scorecard gives you an overview of the machine learning model and the key target values you have set to help all stakeholders determine if your model is ready to be deployed.

![Alt text](Media/rai-scorecard-pdf.png)

The data explorer segment shows you characteristics of your data, as any model story is incomplete without the right understanding of data.

![Alt text](Media/rai-scorecard-pdf-data.png)

The model performance segment displays your model’s most important metrics and characteristics of your predictions and how well they satisfy your desired target values.

![Alt text](Media/rai-scorecard-pdf-performance.png)

Next, you can also view the top performing and worst performing data cohorts and subgroups that are automatically extracted for you to see the blind spots of your model.

![Alt text](Media/rai-scorecard-pdf-cohorts.2.png)

Then you can see the top important factors impacting your model predictions, which is a requirement to build trust with how your model is performing its task. You can further see your model fairness insights summarized and inspect how well your model is satisfying the fairness target values you had set for your desired sensitive groups.

![Alt text](Media/rai-scorecard-pdf-features.png)

![Alt text](Media/rai-scorecard-pdf-features-2.png)

![Alt text](Media/rai-scorecard-pdf-features-2.1.png)

## Conclusion

Currenlty, the Responsible AI dashboard in Azure Machine Learning, is generatable via a variety of experiences through CLI, SDK, or a no-code UI experience. You can also generate the **Responsible AI scorecard**, a PDF report you can extract and share with summaries of key data and model performance and fairness insights. The dashboard supports the ML professional personas, empowering them to easily debug and improve their machine learning models, while the scorecard helps technical and non-technical audiences alike understand the impact of applying Responsible AI, so that all stakeholders can participate in compliance reviews.